In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Import datasets

In [2]:
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [3]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# Prepare training set and test sets

### Training Set

In [6]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')

In [7]:
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [8]:
training_set = np.array(training_set, dtype='int')

In [9]:
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

### Test Set

In [10]:
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')

In [11]:
test_set.head()

,1,6,5,887431973
0,1,10,3,875693118
1,1,12,5,878542960
2,1,14,5,874965706
3,1,17,3,875073198
4,1,20,4,887431883


In [12]:
test_set = np.array(test_set, dtype='int')

In [13]:
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

# Getting the number of users and movies

In [14]:
nb_users = max(max(training_set[:, 0]), max(test_set[:, 0]))
nb_users

943

In [15]:
nb_movies = max(max(training_set[:, 1]), max(test_set[:, 1]))
nb_movies

1682

# Converting the data into an array with users in lines and movies in columns

In [16]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [17]:
training_set = convert(training_set)

In [18]:
test_set = convert(test_set)

# Converting the data into Torch Tensors

In [19]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Creating the architechture of the Neural Networks

In [20]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [21]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

# Training the SAE

In [22]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(1.7708)
epoch: 2 loss: tensor(1.0967)
epoch: 3 loss: tensor(1.0535)
epoch: 4 loss: tensor(1.0383)
epoch: 5 loss: tensor(1.0308)
epoch: 6 loss: tensor(1.0265)
epoch: 7 loss: tensor(1.0236)
epoch: 8 loss: tensor(1.0219)
epoch: 9 loss: tensor(1.0209)
epoch: 10 loss: tensor(1.0196)
epoch: 11 loss: tensor(1.0188)
epoch: 12 loss: tensor(1.0182)
epoch: 13 loss: tensor(1.0179)
epoch: 14 loss: tensor(1.0175)
epoch: 15 loss: tensor(1.0171)
epoch: 16 loss: tensor(1.0169)
epoch: 17 loss: tensor(1.0167)
epoch: 18 loss: tensor(1.0165)
epoch: 19 loss: tensor(1.0162)
epoch: 20 loss: tensor(1.0161)
epoch: 21 loss: tensor(1.0160)
epoch: 22 loss: tensor(1.0158)
epoch: 23 loss: tensor(1.0159)
epoch: 24 loss: tensor(1.0158)
epoch: 25 loss: tensor(1.0157)
epoch: 26 loss: tensor(1.0154)
epoch: 27 loss: tensor(1.0153)
epoch: 28 loss: tensor(1.0149)
epoch: 29 loss: tensor(1.0130)
epoch: 30 loss: tensor(1.0120)
epoch: 31 loss: tensor(1.0098)
epoch: 32 loss: tensor(1.0095)
epoch: 33 loss: t

# Testing the SAE

In [23]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = Variable(test_set[id_user])
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
#             output[target == 0] = 0
            output[(target == 0).unsqueeze(0)] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            test_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
print('loss: '+str(test_loss/s))

/anaconda/envs/py35/lib/python3.5/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1682])) that is different to the input size (torch.Size([1, 1682])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss: tensor(0.9515)
